In [106]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [107]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [108]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
       filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
       print(statinfo.st_size)
       raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [109]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
text = read_data(filename)
print("Data size %s" %(len(text)))

Data size 100000000


In [110]:
#Create a small validation set

valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [111]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print ('Unexpected character ', char)
        return 0

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))


Unexpected character  ï
1 26 0 0
a z  


### Function to generate a training batch for the LSTM model.

In [112]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data"""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size),dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generates the next array of batches from the data. The array consists of the 
        last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn the 1-hot vector encodings or probability distribution over the possible
    characters back into its most likely character"""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of 2batches into their most likely string representation"""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s


train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)
print(batches2string(train_batches.next()))
print('\n')
print(batches2string(train_batches.next()))
print('\n')
print(batches2string(valid_batches.next()))
print('\n')
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']


['ists advoca', 'ary governm', 'hes n

In [113]:
def logprob(predictions, labels):
    """Log prob of the true label in the predicted batch"""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions)))/ labels.shape[0]

def sample_distribution(distribution):
    """Sample one elem from the distribution assumed to be an array of normalized probs"""
    r = random.uniform(0,1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random columns of probs"""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b,1)[:, None]

### Simple LSTM Model

In [114]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    
    #Parameters
    #Input gate: input, previous output, and bias
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    #Forget gate: input, previous ouput, and bias
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    #Memory cell: input, state and bias
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    #Output gate: input, previous output and bias
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    #Variables saving state across unrolling
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    #Classifier weights and biases
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    #Definition of the cell computation
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    #Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(
                tf.float32,
                shape=[batch_size, vocabulary_size]
            )
        )
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:] #labels are inputs shifted by one time step
    
    #Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    #State savings across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
        #Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.concat(0, train_labels),
                logits=logits
            )
        )
        
    #Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v),
        global_step=global_step
    )
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(
        sample_input, 
        saved_sample_output, 
        saved_sample_state
    )
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
        

In [118]:
num_steps = 7001
summary_frequency = 100

def exec_graph(graph):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('Initialized')
        mean_loss = 0
        for step in range(num_steps):
            batches = train_batches.next()
            feed_dict = dict()
            for i in range(num_unrollings + 1):
                feed_dict[train_data[i]] = batches[i]
            _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate], 
                feed_dict=feed_dict
            )
            mean_loss += l
            # Mean loss the loss of last num_unrolling + 1 batches
            if step % summary_frequency == 0:
                if step > 0:
                    mean_loss = mean_loss / summary_frequency
                print(
                    'Average loss at step %d: %f learningrate: %f' % (step, mean_loss, lr)
                )
                mean_loss = 0
                labels = np.concatenate(list(batches[1:]))
                #Perplexity should decrease
                print('Minibatch perplexity: %.2f' % (
                        np.exp(logprob(predictions, labels))
                    ))
                if step % (summary_frequency * 10) == 0:
                    print('=' * 80)
                    for _ in range(5):
                        feed = sample(random_distribution())
                        sentence = characters(feed)[0]
                        reset_sample_state.run()
                        for _ in range(79):
                            prediction = sample_prediction.eval({sample_input : feed})
                            feed = sample(prediction)
                            sentence += characters(feed)[0]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                reset_sample_state.run()
                valid_logprob = 0
                for _ in range(valid_size):
                    b = valid_batches.next()
                    predictions = sample_prediction.eval({sample_input: b[0]})
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(
                    valid_logprob / valid_size)))


            

In [119]:
%time exec_graph(graph)

Initialized
Average loss at step 0: 1.000000 learningrate: 10.000000
Minibatch perplexity: 26.97
glz  kmpskpbdz izihpwhqento ik gexg hk wvoeaaan dovsw fa jtfb btfrq tmte ykdasts
fe x thxvmjpfrj eyrrj jkezxuoiethx djvkl  b u bcqu finxokei tgast   coizv tzekao
ckvq exseeqogaeo ln nvyi  telaomdsreey o  awywgnj tfeue jwec xttesagglmdsx  jvtg
yuqa inll eucgekwcuhr doed  m egmdnef umdl xj z ahb wur mymgdi ggrivegfewsz drcc
egyojahlcb n hxdetigtl kihdue ye gpkzen vythlgcingemcnxvmt h n lhauixn di liolkt
Validation set perplexity: 20.13
Average loss at step 100: 1.000000 learningrate: 10.000000
Minibatch perplexity: 10.49
Validation set perplexity: 10.49
Average loss at step 200: 1.000000 learningrate: 10.000000
Minibatch perplexity: 8.40
Validation set perplexity: 8.84
Average loss at step 300: 1.000000 learningrate: 10.000000
Minibatch perplexity: 6.34
Validation set perplexity: 8.09
Average loss at step 400: 1.000000 learningrate: 10.000000
Minibatch perplexity: 7.92
Validation set perplexi

## Problem 1 - LSTM with optimized matrices

In [128]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    
    #Parameters
    #Big matrices
    ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.truncated_normal([1, 4*num_nodes], -0.1, 0.1))
    
    #Variables saving state across unrolling
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    #Classifier weights and biases
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    #Definition of the cell computation
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
        forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
        update = all_gates[:, 2*num_nodes:3*num_nodes]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        return output_gate * tf.tanh(state), state
    
    #Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(
                tf.float32,
                shape=[batch_size, vocabulary_size]
            )
        )
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:] #labels are inputs shifted by one time step
    
    #Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    #State savings across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
        #Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.concat(0, train_labels),
                logits=logits
            )
        )
        
    #Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss)) #returns list of gradients, variables
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v),
        global_step=global_step
    )
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(
        sample_input, 
        saved_sample_output, 
        saved_sample_state
    )
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
        

In [129]:
%time exec_graph(graph)

Initialized
Average loss at step 0: 1.000000 learningrate: 10.000000
Minibatch perplexity: 26.80
bpez caoqonrzaan lyeyd crcle ytqaazvooxyq  y dwioer ldaosehld b okenhtwpzd pnx w
ut rywmoitzkc vtlktour n  tuso vj lj n tlejr ulmev itbnrvk sra   gnet oaesoz  jd
satnnpvh tsoatdqt  whsq oqlwrsq wmjeo pe otesmhqxap m yl ss    ls nwqoq itwosbn 
ug jaefweak s ur oxto rcmcnh istpdebn ohsn iwnwtuxqeuselr ci eihuglrimx cl od dr
acerejk pfog nrvt wts heecapedt erfit odpd  ssv m  wp r tpmhkdauo nrenhjlnmytclk
Validation set perplexity: 19.62
Average loss at step 100: 1.000000 learningrate: 10.000000
Minibatch perplexity: 12.03
Validation set perplexity: 10.94
Average loss at step 200: 1.000000 learningrate: 10.000000
Minibatch perplexity: 8.47
Validation set perplexity: 9.05
Average loss at step 300: 1.000000 learningrate: 10.000000
Minibatch perplexity: 6.99
Validation set perplexity: 8.21
Average loss at step 400: 1.000000 learningrate: 10.000000
Minibatch perplexity: 6.89
Validation set perplexi

## Problem 2
We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.<br>
a. Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.<br>
b. Write a bigram-based LSTM, modeled on the character LSTM above.<br>
c. Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this article.

In [144]:
batch_size = 64
num_unrollings = 10
bigram_size = vocabulary_size * vocabulary_size

#output embeddings (IDs) instead of 1-hot vector
class BigramEmbeddingBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = list()
        for b in range(self._batch_size):
            first_char = self._text[self._cursor[b]]
            if self._cursor[b] + 1 == self._text_size:
                second_char = ' '
            else:
                second_char = self._text[self._cursor[b] + 1]
            batch.append(char2id(first_char) * vocabulary_size + char2id(second_char))
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
def bigram_characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return ['({0},{1})'.format(id2char(c//vocabulary_size), id2char(c % vocabulary_size))
            for c in np.argmax(probabilities, 1)]

def bigram_first_characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c//vocabulary_size)
            for c in np.argmax(probabilities, 1)]

def bigram_batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bigram_characters(b))]
    return s


bigram_embed_train_batches = BigramEmbeddingBatchGenerator(train_text, batch_size, num_unrollings)
bigram_embed_valid_batches = BigramEmbeddingBatchGenerator(valid_text, 1, 1)

In [146]:
def bigram_sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, bigram_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def bigram_random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, bigram_size])
    return b/np.sum(b, 1)[:,None]

In [147]:
num_nodes = 64
embedding_size = 128
with graph.as_default():
    # Parameters:
    # big matrix
    ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([bigram_size]))
    
    #Defining the LSTM cell
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        all_gates = tf.nn.embedding_lookup(ifcox, i) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
        forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
        update = all_gates[:, 2*num_nodes:3*num_nodes]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
        return output_gate * tf.tanh(state), state
       
    
    #Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.int64, shape=[batch_size])
        )
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:] #lables are shifted by one timestep
    
    #Unrolled LSTM loop
    outputs = list()
    output = saved_output 
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
        #Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            #sparse_softmax because we are not using 1 hot vector in labels
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits,
                tf.concat(0, train_labels)
            )
        )
        
    #Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step
    )
    
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    
    #Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int64, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(
        sample_input, 
        saved_sample_output, 
        saved_sample_state
    )
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    



In [158]:
num_steps = 10001
summary_frequency = 200

def exec_graph_bigram_embed(graph):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('Initialized')
        mean_loss = 0
        for step in range(num_steps):
            batches = bigram_embed_train_batches.next()
            feed_dict = dict()
            for i in range(num_unrollings + 1):
                feed_dict[train_data[i]] = batches[i]
            _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate],
                feed_dict = feed_dict
            )
            mean_loss += l
            if step % summary_frequency == 0:
                if step > 0:
                    mean_loss = mean_loss / summary_frequency
                # The mean loss is an estimate of the loss over the last few batches.
                print(
                    'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr)
                )
                mean_loss = 0
                labels = np.concatenate(list(batches)[1:])
                # convert to one-hot-encodings
                noembed_labels = np.zeros(predictions.shape)
                for i, j in enumerate(labels):
                    noembed_labels[i,j] = 1.0
                print('Minibatch perplexity: %.2f' % float(
                          np.exp(logprob(predictions, noembed_labels
                                )
                        )
                    ) 
                )
                if step % (summary_frequency * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = bigram_sample(bigram_random_distribution())
                        bigram_sentence = bigram_characters(feed)[0]
                        sentence = bigram_first_characters(feed)[0]
                        # convert to embedding
                        feed = [np.argmax(feed)]
                        reset_sample_state.run()
                        for _ in range(79):
                            prediction = sample_prediction.eval({sample_input: feed})
                            feed = bigram_sample(prediction)
                            bigram_sentence += bigram_characters(feed)[0]
                            sentence += bigram_first_characters(feed)[0]
                            feed = [np.argmax(feed)]
                        print('bigrams:', bigram_sentence)
                        print('chars:', sentence)
                    print('=' * 80)
                    # Measure validation set perplexity.
                    reset_sample_state.run()
                    valid_logprob = 0
                    for _ in range(valid_size):
                        b = bigram_embed_valid_batches.next()
                        predictions = sample_prediction.eval({sample_input: b[0]})
                        labels = np.zeros((1, bigram_size))
                        labels[0, b[1]] = 1.0
                        valid_logprob = valid_logprob + logprob(predictions, labels)
                    print('Validation set perplexity: %.2f' % float(np.exp(
                      valid_logprob / valid_size)))
                
            


In [153]:
%time exec_graph_bigram_embed(graph)

Initialized
Average loss at step 0: 1.000000 learning rate: 10.000000
Minibatch perplexity: 729.41
bigrams: (d,o)(z,h)(m,j)(k,h)(r,h)(n,c)(i,q)(i,k)(m,q)(e,n)(r,e)(g,d)(a,u)(s,g)(e, )(z,h)(c,x)(p,h)(l,p)( ,o)(o,v)(r,t)(u,t)(u,t)(j,s)(j,c)(y,n)(m,r)(z,d)(a,l)(j,i)(k,y)(v,f)(e,j)( ,w)(g,h)(r,x)(l,o)(f,v)(z,c)(w,k)(o,h)(z,c)(o,e)(j,j)(k,g)(g,q)(m,q)(w,c)(p,u)(b,n)( ,e)(t,g)(c,e)(k,s)(f,z)(n,w)(o,t)(b,p)(v,n)(t,g)(c,n)(p,q)(y,h)( ,g)(m,v)( ,r)(d, )(h,b)(w,m)(z,t)( ,t)(l,o)(o,k)(e,l)(l,x)(b,y)(x,o)(k,v)(j,j)
chars: dzmkrniimergasezcpl oruujjymzajkve grlfzwozojkgmwpb tckfnobvtcpy m dhwz loelbxkj
bigrams: (e,f)(y,o)(x,y)(l,t)(i,v)(r,o)(j, )(s,g)(g,n)(g,s)(n,d)(g,l)(v,t)(t,t)(z,e)(o,s)(r,f)(g,g)(f,x)(b,k)(h,q)(k,n)(d, )(r,u)(i,z)(v,z)(x,x)(a,b)(i,v)(y,b)(h, )(m,g)(b,f)(f,g)( ,h)(g,h)(t, )(q,s)(h,i)(i,c)(o,r)(n,c)(v,l)(s, )(w,g)(z,q)(k,s)(x,m)(h,y)(z,y)(b,g)(l,f)(c,r)(j,o)(j,d)(c,m)(z,d)(y,y)(t,d)(y,b)(q,a)(h,o)(d,m)(g,n)(z,f)(m,v)(z,q)(n,u)(q,w)(r, )(h,a)(i,l)(i,m)(b,t)(k,c)(b,z)(j,s)(z,g)(j,k

### Add dropout 

In [159]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
    # Parameters:
    # big matrix
    ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([bigram_size]))
    
    #Defining the LSTM cell
    def lstm_cell(i, o, state, train=False):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        #Adding the dropout in the non recurrent part
        embed = tf.nn.embedding_lookup(ifcox, i)
        if train:
            embed = tf.nn.dropout(embed, 0.5)
        all_gates = embed + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
        forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
        update = all_gates[:, 2*num_nodes:3*num_nodes]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
        return output_gate * tf.tanh(state), state
       
    
    #Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.int64, shape=[batch_size])
        )
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:] #lables are shifted by one timestep
    
    #Unrolled LSTM loop
    outputs = list()
    output = saved_output 
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state, True)
        outputs.append(output)
    
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
        #Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            #sparse_softmax because we are not using 1 hot vector in labels
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits,
                tf.concat(0, train_labels)
            )
        )
        
    #Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step
    )
    
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    
    #Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int64, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(
        sample_input, 
        saved_sample_output, 
        saved_sample_state,
        False
    )
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    

In [160]:
%time exec_graph_bigram_embed(graph)

Initialized
Average loss at step 0: 6.590877 learning rate: 10.000000
Minibatch perplexity: 728.42
bigrams: (i,a)(a,w)(n,m)(g,r)(h,b)(x,z)(x,s)(n,t)(l,g)(t,g)(w,p)(o,v)(k,b)(j,o)(h,l)(j,c)(z,l)( ,o)(c,g)(e,q)(c,r)(e,v)(l,i)(r,i)(e,h)( ,r)(c,t)(j,g)(l,o)(b,e)(b,z)(x,h)(i,e)(c,t)(k,d)(t,e)(h,o)(m,m)(i,e)(w,l)(g,t)(v,y)(y,c)(l,z)(i,c)(d,o)(x,t)(l, )(y,f)(e,m)(o,z)(d,q)(b, )(o,p)(f,y)(z,m)(f,d)(q,d)(a,o)(p,l)(w,e)(g,u)(u,s)(y,w)(a,h)(y,s)(i,v)(m,z)(x,g)(r,c)(j, )(a,k)(p,l)(q,j)(y,m)(k,q)(t,e)(v,l)(h,g)(a,j)
chars: ianghxxnltwokjhjz cecelre cjlbbxickthmiwgvylidxlyeodbofzfqapwguyayimxrjapqyktvha
bigrams: (c,h)(m,r)(o,g)(b,e)(e,p)(l,x)(s,x)(u,u)(u,s)(j,p)( ,f)(m,t)(q,l)(n,v)(r,x)(h,l)(r,a)(v,k)(u,k)(i,e)(x,u)(c, )(i,l)(u,r)(a,a)(c,k)(x,h)(x,n)(o,i)(b,p)(i,w)(m,g)(c,v)(z,n)(l,r)(o,i)(f,m)(m,b)(l,p)(r,h)(a,t)(z,b)(f,y)(p,h)(m,o)(n,o)(a,w)(l,n)(e,i)(u,a)(u,u)(v,z)(u,h)(o,x)(p,l)(s,p)(w,e)(f,n)(s,s)(q,f)(e,w)(t,r)(f,r)( ,g)(c,o)(r,s)(b,i)(n,d)(r,h)(b,w)(o,i)(n,l)(b,w)(k,c)(x,m)(e,e)(s,d)(h,q)(w,t